In [1]:
# Data/python?Summa.py
import scipy.io
import warnings
import numpy as np
import matplotlib.pyplot as plt

def evaluateSummary(summary_selection,videoName,HOMEDATA):
     '''Evaluates a summary for video videoName (where HOMEDATA points to the ground truth file)   
     f_measure is the mean pairwise f-measure used in Gygli et al. ECCV 2013 
     NOTE: This is only a minimal version of the matlab script'''
     # Load GT file
     gt_file=HOMEDATA+'/'+videoName+'.mat'
     gt_data = scipy.io.loadmat(gt_file)
     
     user_score=gt_data.get('user_score')
     nFrames=user_score.shape[0];
     nbOfUsers=user_score.shape[1];
    
     # Check inputs
     if len(summary_selection) < nFrames:
          warnings.warn('Pad selection with %d zeros!' % (nFrames-len(summary_selection)))
          summary_selection.extend(np.zeros(nFrames-len(summary_selection)))

     elif len(summary_selection) > nFrames:
          warnings.warn('Crop selection (%d frames) to GT length' %(len(summary_selection)-nFrames))       
          summary_selection=summary_selection[0:nFrames];
             
     
     # Compute pairwise f-measure, summary length and recall
     summary_indicator=np.array(map(lambda x: (1 if x>0 else 0),summary_selection));    
     user_intersection=np.zeros((nbOfUsers,1));
     user_union=np.zeros((nbOfUsers,1));
     user_length=np.zeros((nbOfUsers,1));
     for userIdx in range(0,nbOfUsers):
         gt_indicator=np.array(map(lambda x: (1 if x>0 else 0),user_score[:,userIdx]))
         
         user_intersection[userIdx]=np.sum(gt_indicator*summary_indicator);
         user_union[userIdx]=sum(np.array(map(lambda x: (1 if x>0 else 0),gt_indicator + summary_indicator)));         
                  
         user_length[userIdx]=sum(gt_indicator)
    
     recall=user_intersection/user_length;
     p=user_intersection/np.sum(summary_indicator);

     f_measure=[]
     for idx in range(0,len(p)):
          if p[idx]>0 or recall[idx]>0:
               f_measure.append(2*recall[idx]*p[idx]/(recall[idx]+p[idx]))
          else:
               f_measure.append(0)
     nn_f_meas=np.max(f_measure);
     f_measure=np.mean(f_measure);
    
     nnz_idx=np.nonzero(summary_selection)
     nbNNZ=len(nnz_idx[0])
         
     summary_length=float(nbNNZ)/float(len(summary_selection));
       
     recall=np.mean(recall);
     p=np.mean(p);
     
     return f_measure, summary_length


def plotAllResults(summary_selections,methods,videoName,HOMEDATA):
    '''Evaluates a summary for video videoName and plots the results
      (where HOMEDATA points to the ground truth file) 
      NOTE: This is only a minimal version of the matlab script'''
    
    # Get GT data
    gt_file=HOMEDATA+'/'+videoName+'.mat'
    gt_data = scipy.io.loadmat(gt_file)
    user_score=gt_data.get('user_score')
    nFrames=user_score.shape[0];
    nbOfUsers=user_score.shape[1];    

    ''' Get automated summary score for all methods '''
    automated_fmeasure={};
    automated_length={};
    for methodIdx in range(0,len(methods)):
        summaryIndices=np.sort(np.unique(summary_selections[methodIdx]))
        automated_fmeasure[methodIdx]=np.zeros(len(summaryIndices));
        automated_length[methodIdx]=np.zeros(len(summaryIndices));
        idx=0
        for selIdx in summaryIndices:
            if selIdx>0:
                curSummary=np.array(map(lambda x: (1 if x>=selIdx else 0),summary_selections[methodIdx]))    
                f_m, s_l = evaluateSummary(curSummary,videoName,HOMEDATA)
                automated_fmeasure[methodIdx][idx]=f_m
                automated_length[methodIdx][idx]=s_l
                idx=idx+1

    
    ''' Compute human score '''
    human_f_measures=np.zeros(nbOfUsers)
    human_summary_length=np.zeros(nbOfUsers)
    for userIdx in range(0, nbOfUsers):
        human_f_measures[userIdx], human_summary_length[userIdx] = evaluateSummary(user_score[:,userIdx],videoName,HOMEDATA);

    avg_human_f=np.mean(human_f_measures)
    avg_human_len=np.mean(human_summary_length)
    

    ''' Plot results'''
    fig = plt.figure()
    p1=plt.scatter(100*human_summary_length,human_f_measures)
    colors=['r','g','m','c','y']
    for methodIdx in range(0,len(methods)):
        p2=plt.plot(100*automated_length[methodIdx],automated_fmeasure[methodIdx],'-'+colors[methodIdx])
        
    plt.xlabel('summary length[%]')
    plt.ylabel('f-measure')
    plt.title('f-measure for video '+videoName)
    legend=list(methods)    
    legend.extend(['individual humans'])
    plt.legend(legend)
    plt.ylim([0,0.85])
    plt.xlim([0,20])
    plt.plot([5, 5],[0, 1],'--k')
    plt.plot([15.1, 15.1],[ 0, 1],'--k')
    plt.show()

In [3]:
# shot_detect.py

import os, sys, glob

# Get duration of the video file
if os.path.exists('duration.txt'):
    os.remove('duration.txt')

cmd = 'ffmpeg -i input.avi 2>&1|grep "Duration">> duration.txt'
os.system(cmd)

print ' ================= '
print 'Detecting shot boundaries...'
print ' ================= '
if os.path.exists('scenes.txt'):
	os.remove('scenes.txt')

cmd = 'ffprobe -show_frames -of compact=p=0 -f lavfi "movie=input.avi,select=gt(scene\,0.3)">> scenes.txt'
os.system(cmd)


# uncompress the movie file for accurate partition
# skip this if you do not need high accuracy
print ' ================= '
print 'computing raw video for accurate segmentation...'
print ' ================= '
cmd = 'ffmpeg -i input.avi -vcodec rawvideo -acodec copy uncompressed.avi' #pcm_s16le
os.system(cmd)

# read time stamps for keyframes
print ' ================= '
print 'Partitioning video...'
print ' ================= '
seginfo = 'scenes.txt'
if not os.path.exists('scenes'):
	os.mkdir('scenes')

tb = '0'
te = '0'
count = 1
f = open('scenesFinal.txt', 'w')
for line in open(seginfo,'r'):
    line = line.replace("|"," ")
    line = line.replace("="," ")
    parts = line.split()
    te = parts[11] # timestamp
    te = float(te)
    fstr = str(count) + ' ' + str(te) + '\n'
    f.write(fstr)
    cmd = 'ffmpeg -ss '
    tb = float(tb)
    # start time
    if (count == 1):
        tbw = '00:00:00' # first shot
    else:
        tbh = int(tb/3600)
        tbm = int((tb%3600)/60)
        tbs = ((tb%3600)%60)%60
        tbw = str(tbh) + ':' + str(tbm) + ':' + str(tbs)
    cmd += tbw + ' -i uncompressed.avi -vcodec mpeg4 -acodec copy -t ' # change codecs if necessary
    # duration
    td = te - tb
    tdh = int(td/3600)
    tdm = int((td%3600)/60)
    tds = ((td%3600)%60)%60
    tdw = str(tdh) + ':' + str(tdm) + ':' + str(tds)
    cmd += tdw + ' scenes/' + '%(#)04d.avi' % {"#":count}
    os.system(cmd)
    tb = te
    count += 1

os.remove('scenes.txt')
os.remove('uncompressed.avi')

Detecting shot boundaries...
computing raw video for accurate segmentation...
Partitioning video...


WindowsError: [Error 2] The system cannot find the file specified: 'uncompressed.avi'

In [4]:
cmd = 'ffmpeg -i input.avi -vcodec rawvideo -acodec copy uncompressed.avi'
os.system(cmd)

-2

In [2]:
# Scripts/Shot_Boundary/scc.py
def strongly_connected_components_tree(vertices, edges):
    identified = set()
    stack = []
    index = {}
    lowlink = {}

    def dfs(v):
        index[v] = len(stack)
        stack.append(v)
        lowlink[v] = index[v]

        for w in edges[v]:
            if w not in index:
                # For Python >= 3.3, replace with "yield from dfs(w)"
                for scc in dfs(w):
                    yield scc
                lowlink[v] = min(lowlink[v], lowlink[w])
            elif w not in identified:
                lowlink[v] = min(lowlink[v], lowlink[w])

        if lowlink[v] == index[v]:
            scc = set(stack[index[v]:])
            del stack[index[v]:]
            identified.update(scc)
            yield scc

    for v in vertices:
        if v not in index:
            # For Python >= 3.3, replace with "yield from dfs(v)"
            for scc in dfs(v):
                yield scc

In [5]:
# Scripts/Shot_Boundary/key_frame_extraction.py
import sys
import imageio
import numpy as np
import cv2
import scipy.io
import pywt
import os, sys, glob
#from scc import strongly_connected_components_tree

# System Arguments
# Argument 1: Location of the video
# Argument 2: Sampling rate (k where every kth frame is chosed)

# defines the number of bins for pixel values of each type as used the original work
num_bins_H=32
num_bins_S=4
num_bins_V=2

# manual function to generate histogram on HSV values
def generate_histogram_hsv(frame):
	hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	hsv_frame = hsv_frame
	global num_bins_H, num_bins_S, num_bins_V
	hist = cv2.calcHist([frame], [0, 1, 2], None, [256/num_bins_H, 256/num_bins_S, 256/num_bins_V],
		[0, 256, 0, 256, 0, 256])
	hist = cv2.normalize(hist).flatten()
	return hist;

# function to calculate the distance matrix for bhattacharyya_distance
def bhattacharyya_distance(color_histogram):
	distance_matrix=np.zeros((len(color_histogram),len(color_histogram)))
	for i in range(len(color_histogram)):
		temp_list = []
		for j in range(len(color_histogram)):
			if i != j:
				distance_matrix[i][j] = cv2.compareHist(color_histogram[i],color_histogram[j],cv2.cv.CV_COMP_BHATTACHARYYA)
			else:
				distance_matrix[i][j] = float("inf")
	return distance_matrix

def save_keyframes(frame_indices, summary_frames):
	print "Saving frame indices"
	video_name = sys.argv[1]
	video_name = video_name.split('.')
	video_name = video_name[0].split('/')
	out_file=open("frame_indices_"+video_name[1]+".txt",'w')
	for idx in frame_indices:
		out_file.write(str(idx)+'\n')
	print "Saved indices"

	# print "Saving frames"
	# for i,frame in enumerate(summary_frames):
	# 	cv2.imwrite(str(sys.argv[7])+"keyframes/frame%d.jpg"%i, frame)
	# print "Frames saved"


def main():
	if len(sys.argv) < 2:
		print "Incorrect no. of arguments, Halting !!!!"
		return
	print "Opening video!"

	video = imageio.get_reader(sys.argv[1], 'ffmpeg')
	print "Video opened\nChoosing frames"

	if len(sys.argv) >=3:
	#frame chosen every k frames
		print "choosing frames uniformly"
		sampling_rate=int(sys.argv[2])
		frames=[np.array(video.get_data(i*sampling_rate)) for i in range(len(video)/sampling_rate)]

	else:
		# delete scenes.txt if it already exists
		print "Detecting different shots"
		if os.path.exists('scenes.txt'):
			os.remove('scenes.txt')
		# use the parameter currently set as "0.4" to control the no. of frames to be selected
		video_name = sys.argv[1]
		video_name = video_name.split('/')[-1]
		print video_name
		os.chdir('input')
		cmd = 'ffprobe -show_frames -of compact=p=0 -f lavfi "movie='+str(video_name)+',select=gt(scene\,0.1)">> ../scenes.txt'
		os.system(cmd)
		os.chdir('..')
		seginfo = 'scenes.txt'
		frame_index_list = []
		for line in open(seginfo,'r'):
			line = line.replace("|"," ")
			line = line.replace("="," ")
			parts = line.split()
			frame_index_list.append(int(parts[11])) #appending the frame no. in the list of selected frames
		print frame_index_list, len(video)
		frames = []
		for i in range(len(frame_index_list)):
			if frame_index_list[i] >= 0 and frame_index_list[i] < len(video):
				frames.append(np.array(video.get_data(frame_index_list[i])))

	if len(frames) <= 0:
		print "unable to detect any shot, Halting !!!!"
		return
	print "Frames chosen: ",len(frame_index_list)
	#extracting color features from each representative frame
	print "Generating Histrograms"
	color_histogram=[generate_histogram_hsv(frame) for frame in frames]
	print "Color Histograms generated"

	#to-do (optional): extract texture features for each frame

	#calculate distance between each pair of feature histograms
	print "Evaluating the distance matirix for feature hitograms"
	distance_matrix = bhattacharyya_distance(color_histogram)
	print "Done Evalualting distance matrix"

	#constructing NNG (nearest neighbour graph) based of distance_matrix
	print "Constructing NNG"
	eps_texture_NN = [None]*len(distance_matrix[0])
	for i in range(0,len(distance_matrix[0])):
		temp = float(0)
		for j in range(len(distance_matrix[i])):
			if distance_matrix[i][j] >= temp:
				eps_texture_NN[i] = j
				temp = distance_matrix[i][j]

	#constructing RNNG(reverse nearest neighbour graph) for the above NNG
	print "Constructing RNNG"
	eps_texture_RNN = {}
	for i in range(len(eps_texture_NN)):
		if eps_texture_NN[i] in eps_texture_RNN.keys():
			eps_texture_RNN[eps_texture_NN[i]].append(i)
		else:
			eps_texture_RNN[eps_texture_NN[i]] = [i]
		if i not in eps_texture_RNN.keys():
			eps_texture_RNN[i] = []

	#calculating the SCCs(strongly connected components) for RNNG
	print "Finiding the strongly connected components of RNNG"
	vertices = [i for i in range(0,len(frames))]
	scc_graph = strongly_connected_components_tree(vertices, eps_texture_RNN)

	#choosing one frame per SCC in summary
	print "Evaluating final summary"
	summary = []
	summary_frames = []
	skim_length = 40
	for scc in scc_graph:
		frame_to_add = frame_index_list[next(iter(scc))]
		for i in range(-skim_length,skim_length):
			if frame_to_add + i > 0 and frame_to_add + i < len(video):
				if frame_to_add+i not in summary:
					summary.append(frame_to_add+i)
				# summary_frames.append(video.get_data(frame_to_add + i))

	# writing the summary in a file 
	os.chdir('summary')
	save_keyframes(summary, summary_frames)

if __name__ == '__main__':
	main()

Opening video!


IOError: No such file: 'C:\Users\HP\py2env\vsumm\Scripts\Shot_Boundary\-f'

In [6]:
# evaluate.py

import sys
sys.path.append("../../Data/python")
import os
from summe import *
import imageio
# System Arguments
# Argument 1: Location of the video
# Argument 2: Location of predicted summary
# Argument 3: Location to store results : typically of the form 'result_"video_name".txt'

def main():
	video=sys.argv[1]
	video_name = sys.argv[1]
	video_name = video_name.split('/')
	video_name = video_name[-1].split('.')[0]
	print "video_name ", video_name
	directory=sys.argv[2]

	print "Getting frames of summary!"
	frame_indices=[int(idx) for idx in open(directory+'/frame_indices_'+video_name+'.txt','r').read().splitlines()]
	print "Got the frames'"

	video=video.split('/')
	videoName=video[len(video)-1].split('.')[0]
	print videoName
	
	video[len(video)-2]="GT"
	HOMEDATA='/'.join(video[0:len(video)-1])
	print HOMEDATA
	
	f_measure, summary_length=evaluateSummary(frame_indices,videoName,HOMEDATA)
	print "F-measure %.3f at length %.2f" %(f_measure, summary_length)

	out_file=open(sys.argv[3],'a')
	out_file.write("F-measure, Summary Length\n")
	out_file.write("%f,%f\n"%(f_measure,summary_length))

if __name__ == '__main__':
	main()

video_name  -f
Getting frames of summary!


IOError: [Errno 2] No such file or directory: 'C:\\Users\\HP\\AppData\\Roaming\\jupyter\\runtime\\kernel-86821c81-e08c-4d46-8bf3-eccad2714221.json/frame_indices_-f.txt'